# Наследование. Работа с вебом

Алексей Умнов https://www.youtube.com/watch?v=0wgAZL9yV0I  
Слайды доступны по адресу: http://parallels.nsu.ru/~fat/Python/

## Доступ к переопределенным методам

В Python наследуются только методы от родительского класса, атрибуты объекта созданного от родительского класса унаследовать потомком нельзя (что вообщем-то логично):

In [3]:
class A():
    
    def __init__(self):
        # Создается атрибут у объекта, а не класса!!!
        # Атрибут пренадлежит объекту вызвавшего этот метод
        self.x = 'a'


class B(A):
    
    def __init__(self):
        # В Python версии 3.x в круглых скобках super() можно не писать класс, он сам его определяет автоматически
        super(B, self).__init__() # Вызывается метод инициализации экземпляра класса (не класса!)
        # A.__init__(self)        # Нежелаетельный способ
        self.y = 'b'              # Создается атрибут у объекта, а не класса!!!



b = B()

print(b.x)
print(b.y)

a
b


Дополнение методов (extend):

In [4]:
class A():
    
    def f(self, a, b):
        return a + b


class B(A):

    # Это переопределенный метод. В Python не существует перегрузки методов!
    # Расширяется (совершенствуется) базовый метод 'f' родительского класса 'A':
    def f(self, a, b):
        return super(B, self).f(a, b) * 2
        # return super().f(a, b) * 2 # Аналогично для Python версии > 3.х



b = B()
print(b.f(1, 2))

6


## Множественное наследование

#### Ромбовидное наследование

In [5]:
class X():
    pass

class A(X):
    pass

class B(X):
    pass

class C(A, B):
    pass

Порядок разрешения имен? Ответ:

**MRO** - **M**ethod **R**esolution **O**rder

Используется алгоритм C3-линеаризации. В случае ромбовидного наследования, порядок получается следующий: 
````
C0 -> A1 -> B2 -> X3
````

В Python существует метод `mro()`, который отображает иерархию наследования, порядок в котором методы классов будут вызываться:

In [6]:
class F():
    pass

class E():
    pass

class D():
    pass

class C(D, F):
    pass

class B(D, E):
    pass

class A(B, C):
    pass


A.mro()

[__main__.A,
 __main__.B,
 __main__.C,
 __main__.D,
 __main__.E,
 __main__.F,
 object]

Пример, доступ к переопределенным методам:

In [7]:
class C():
    
    def f(self, a, b):
        print('Method of C class')
        return a + b


class B():
    
    def f(self, a, b):
        print('Method of B class')
        return a + b


class A(B, C): # Важность порядка наследования
    
    def f(self, a, b):
        # Python сам определяет к какому методу какого класса обратиться по порядку наследования классов.
        return super().f(1, 2) # Сначала, вызываемый метод будет искаться в классе B, а затем в классе C



a = A()
a.f(1, 2)

Method of B class


3

## Работа с интернет-ресурсами

Библиотека `urllib` для Python v3.x, для Python v2.x - `urllib2`

In [12]:
from urllib.request import urlopen

with urlopen('https://yandex.ru') as conn:
    data = conn.read()
    html = data.decode('utf-8') # Из байтов получаем unicode текст
    print('real_url =', conn.geturl())
    # print('conn_info =', conn.info())
    print('http_code =', conn.getcode())
    print('data = ', html[:500])

real_url = https://yandex.ru
http_code = 200
data =  <!DOCTYPE html><html class="i-ua_js_no i-ua_css_standart i-ua_browser_ i-ua_browser_desktop i-ua_platform_other" lang="ru"><head xmlns:og="http://ogp.me/ns#"><meta http-equiv="X-UA-Compatible" content="IE=edge"><title>Яндекс</title><link rel="shortcut icon" href="//yastatic.net/iconostasis/_/8lFaTHLDzmsEZz-5XaQg9iTWZGE.png"><meta http-equiv=Content-Type content="text/html;charset=UTF-8"><link rel="apple-touch-icon" href="//yastatic.net/iconostasis/_/5mdPq4V7ghRgzBvMkCaTzd2fjYg.png" sizes="76x76"


Кодирование ссылок:

In [13]:
from urllib.parse import urlencode

try:
    url = urlencode((('id', 1), ('q', 'hello!')))  # можно передать кортеж кортежей
    url = urlencode({'id': 1, 'q': 'hello!'})      # или словарь
    print(url)                                     # id=1&q=hello%21
except TypeError as e:
    print('Exception:', e)

id=1&q=hello%21


## Обработка XML и HTML

Дерево элементов HTML:

In [ ]:
# %load sample.html
<html>
    <head>
        <title>Page title</title>
    </head>
    <body>
        <b>important</b>
        <br />
        <p id="main">
            Some text
            Text line 2
        </p>
    </body>
</html>


Библиотека `xml.etree.ElementTree`:

In [23]:
import xml.etree.ElementTree as ET

tree = ET.parse('sample.html')
root = tree.getroot()

print(len(root), type(root))  # len = 2 элемента, тип = <class 'xml.etree.ElementTree.Element'>

# Этот тип умеет притворятся быть списком через переопределение квадратных скобок:
par = root[1][2]       # <body> --> <p>
print(type(par))       # <class 'xml.etree.ElementTree.Element'>
print(par.tag)         # p
print(par.attrib)      # {'id': 'main'}
print(par.text)        # Some text

2 <class 'xml.etree.ElementTree.Element'>
<class 'xml.etree.ElementTree.Element'>
p
{'id': 'main'}

            Some text
            Text line 2
        


Еще пример обхода дерева элементов:

In [24]:
body = None

for child in root:     # Так как объект root может претворятся списком, то мы можем пройтись по нему циклом
    if child.tag == 'body':
        body = child   # child это объект
        break

texts = []

for child in body:     # Цикл по объектам в объекте body
    if(child.tag == 'p' and 'id' in child.attrib and child.attrib['id'] == 'main'):
        texts.append(child.text)

print(texts)

['\n            Some text\n            Text line 2\n        ']


Но так делать, как выше не обязательно, есть специальный язык запросов дерева XML, это XPath:

In [25]:
print(root.findall('.')) # Текущее поддерево

[<Element 'html' at 0x7f55ec0494a8>]


Все элементы, которые можно получить из текущего поддерева, встретив на пути теги `<body>` и `<b>`:

In [27]:
print(root.findall('./body/b'))

[<Element 'b' at 0x7f55ec0497c8>]


Все элементы, которые можно получить из текущего поддерева, встретив на пути любые теги (//), а затем теги `<p>` и `<span>`:

In [28]:
print(root.findall('.//p/span'))

[]


Все элементы, которые можно получить из текущего поддерева, встретив на пути любые теги (//), а затем теги `<p>` и с атрибутом `id` равным `main`:

In [29]:
print(root.findall('.//p[@id="main"]'))

[<Element 'p' at 0x7f55ec049868>]


`xml.tree` плохо работает со "сломанным" HTML (Почти любая HTML-страница - "сломанный" XML), т.е. могут быть не закрыты теги или что-то где-то не так поставили и `xml.tree` такое не допускает, он работает только с правильным кодом.

Альтернативы (не в стандартной библиотеке):
* **lxml** - lxml.html имеет тот же синтакис, что и xml.tree, но может работать с HTML
* **BeautifulSoup**

## HTML Parser

Предыдущие примеры нежелательно использовать для больших документов XML, так как они для того чтобы построить дерево, загружают весь документ в память. Для этого есть библиотеки которые не загружают его полностью в память, а
последовательно просматривают его какие-то части:

````python
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):

   # HTMLParser бежит по документу и вызывает события соответствующие переопределенным методам handle_*
   def handle_starttag(self, tag, attrs):
       print('Start tag:', tag, attrs)

   def handle_endtag(self, tag):
       print('End tag:', tag)
   
   def handle_data(self, data):
       text = data.strip()
       if text:
           print('Data:', text)


parser = MyHTMLParser()
parser.feed(html)
````

Пример программы:

````python
from html.parser import HTMLParser

class MainTextParser(HTMLParser):

   def __init__(self):
       super().__init__(self)
       self.main_text = False
   
   # HTMLParser бежит по документу и вызывает события соответствующие переопределенным методам handle_*
   def handle_starttag(self, tag, attrs):
       if(tag == 'p' and any([(k,v) == ('id', 'main') for k, v in attrs])):
           self.main_text = True
   
   def handle_endtag(self, tag):
       if self.main_text and tag == 'p':
           self.main_text = False
   
   def handle_data(self, data):
       if self.main_text:
           print(data)
````